# Organize datasets

In [1]:
%load_ext autoreload
%autoreload 2

from utils.prediction_models import *

This notebook documents processing of the three datasets:

1. Dataset 1: Zeqian 

    Datast 1 consists of strains from three sources: 
    - [Mario](https://www.nature.com/articles/s41396-019-0427-7): assembly downloaded from NCBI
    - [Karna](https://pubmed.ncbi.nlm.nih.gov/35085485/): assembly provided by Karna 
    - [Kaumudi](https://www.biorxiv.org/content/10.1101/2022.06.27.497809v1): newly sequenced. Trimmed with Trimmomatic and assembled by Unicycler. 
    
    Dataset 1 goes through a snakemake KO annotation + 16S annotation + FBA snakemake pipeline. Utilization data is from new experiments. 

2. Dataset 2: [Matti](https://www.biorxiv.org/content/10.1101/2022.08.04.502823v1.abstract)
    
    Genomes and utilization data are organized from supplementary materials. 16S sequences are annotated from contigs using Barrnap. 
3. Dataset 3: [BacDive](https://bacdive.dsmz.de/)

    Utilization data and accession numbers are downloaded from BacDive website. Genomes eggnog annotations are from [ProGenomes](https://progenomes.embl.de/) by accession ID. 16S sequences are downloaded from ENA database by accession ID.

4. Merged data: merged dataset 1, dataset 2, and dataset 3.  

All finalized datasets stored in the same format:

- A Python dictionary with these keys:
    - 'ko_data': presence-absence KO matrix, with index being samples and columns being kos.
    - 'growth_data: binary growth matrix, with index being samples and columns being carbons. 
    - 'tree': a ete3.Tree object of phylogenetic tree. Note that the tree should be cleaned (no empty leaves). 
    - 'samples': an array of samples. Note that ko_data, growth_data, and tree should have the same samples. Samples with missing data are eliminated. 
    - 'carbons': an array of carbons. 
- The dictionary is saved as a pickle object. 
- (Maybe): save a text format of all these object. This is only for publishing and not used in any data processing. 


In [2]:
# Utility functions for all three datasets
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from Bio import SeqIO
from genomics_utils import IO


def concatenate_barrnap_output(DIR_barrnap, FILE_out,prefix=None):
    missing_16S=[]
    records=[]

    samples=[ff.replace(".16S.fna","") for ff in os.listdir(DIR_barrnap) if ff.endswith("fna")]
    for sample in tqdm(samples):
        df=IO.read_fasta(os.path.join(DIR_barrnap,sample+".16S.fna"),df=True)
        if len(df)<=0:
            missing_16S.append(sample)
            continue
        df_16S=df[df['description'].str.contains('16S')]
        if len(df_16S)<=0:
            missing_16S.append(sample)
        else:
            line=df_16S.iloc[0]
            id_=sample
            if prefix is not None:
                id_=prefix+"_"+sample
            records.append(SeqRecord(seq=Seq(line['seq']), id=id_, description=line['description']))

    print(f"{len(missing_16S)} samples miss 16S sequences: ",missing_16S)

    with open(FILE_out, "w") as output_handle:
        SeqIO.write(records, output_handle, "fasta")

def generate_tree(INPUT, OUTPUT_ALIGNED, OUTPUT_CSV, TREE, threads=1,
        SILVA_DB="/home/zeqianli/scratch-midway3/bin/silva/SILVA_138.1_SSURef_opt.arb"):
    !/home/zeqianli/scratch-midway3/miniconda3/envs/sixteenS/bin/sina -i {INPUT} -o {OUTPUT_ALIGNED} -o {OUTPUT_CSV} -r {SILVA_DB} --search --search-db {SILVA_DB} --lca-fields tax_slv,tax_embl_ebi_ena,tax_embl_ebi_ena_name,tax_gtdb --threads {threads}
    !/home/zeqianli/scratch-midway3/miniconda3/envs/sixteenS/bin/fasttree -nt -gtr -gamma -out {TREE} {OUTPUT_ALIGNED}

In [ ]:
def finalize_data(ko_data, growth_data, tree,remove_prefix=False):    
    samples=np.intersect1d(ko_data.index,growth_data.index)
    tree_samples=[leaf.name for leaf in tree.get_leaves()]
    tree=tree.copy()
    if remove_prefix is not None:
        for leaf in tree.get_leaves():
            leaf.name=leaf.name.replace("matti_","").replace("zeqian_","").replace("bacdive_","")
    
    samples=np.intersect1d(samples,[node.name for node in tree.get_leaves()])
    print(f"{len(samples)} samples: ", samples)

    ko_data=ko_data.loc[samples]
    growth_data=growth_data.loc[samples]
    tree.prune(samples, preserve_branch_length=True)
    return {'ko_data':ko_data, 'growth_data':growth_data,'tree':tree,'samples': samples, 'carbons':growth_data.columns.values}


# Dataset 1: Zeqian (this study)

In [3]:
DIR_zeqian="/home/zeqianli/project/zeqian/Carbon/data/zeqian"

## 1.1. Organize assemblies

In [ ]:
# Mario 

ff_mario_ncbi="/home/zeqianli/Kuehn/Carbon/data/mario_ncbi_assembly.txt"
DIR_mario_raw="/home/zeqianli/scratch-midway2/Carbon/raw/Mario"
DIR_assembly="/home/zeqianli/scratch-midway2/Carbon/assembly"
df=pd.read_csv(ff_mario_ncbi,sep=",",header=None)

for _, name, acc, __ in tqdm(df.itertuples()):
    #!/home/zeqianli/project/zeqianli/miniconda3/envs/ncbi/bin/datasets download genome accession {acc} --filename {os.path.join(DIR_mario_raw, name+'.zip')}
    #!unzip -o -q {os.path.join(DIR_mario_raw, name+'.zip')} -d {os.path.join(DIR_mario_raw, name)}
    !cp {os.path.join(DIR_mario_raw, name)}/ncbi_dataset/data/{acc}/{acc}* {os.path.join(DIR_assembly, name+'.fasta')}

In [ ]:
# Karna
import shutil, os, tqdm

DIR_raw_karna="/home/zeqianli/project/zeqianli/Kuehn/Carbon/data/zeqian/raw/Karna"
DIR_assembly="/home/zeqianli/project/zeqianli/Kuehn/Carbon/data/zeqian/assembly"


for ff in tqdm.tqdm([ff for ff in os.listdir(DIR_raw_karna) if ff.endswith(".fasta")],desc="Copying Karna assemblies"):
    shutil.copyfile(os.path.join(DIR_raw_karna, ff), os.path.join(DIR_assembly, ff.replace("_contigs.fasta",".fasta")))

In [ ]:
# Kaumudi 

import shutil
DIR_raw_kaumudi="/home/zeqianli/scratch-midway2/Carbon/raw/Kaumudi"
DIR_assembly="/home/zeqianli/scratch-midway2/Carbon/assembly"

for ff in tqdm(os.listdir(DIR_raw_kaumudi),desc="Copying Kaumudi assemblies"):
    shutil.copyfile(os.path.join(DIR_raw_kaumudi, ff, 'assembly.fasta'), os.path.join(DIR_assembly, ff+'.fasta'))

## 1.2 Downstream processing

A snakemake pipeline does the following steps:
1. KO annotation: assemblies -> prodigal -> kofamscan -> ko files for each samples
2. 16S: assemblies -> barnnap -> rRNA sequences for each sample
3. FBA: assemblies -> prodigal -> carveme -> custom FBA simulation script -> FBA predicted growth rate on carbons for each sample

Then I concatenate kos, 16S, and FBA outputs here, format data, and pickle it. 

In [ ]:
concatenate_barrnap_output(f"{DIR_zeqian}/16S", f"{DIR_zeqian}/zeqian_16S.fna",prefix="zeqian",)

In [ ]:
SILVA_DB="/home/zeqianli/scratch-midway3/bin/silva/SILVA_138.1_SSURef_opt.arb"

!/home/zeqianli/scratch-midway3/miniconda3/envs/sixteenS/bin/sina --arb-list-fields {SILVA_DB}

In [4]:
INPUT=f"{DIR_zeqian}/zeqian_16S.fna"
OUTPUT_ALIGNED=f"{DIR_zeqian}/zeqian_16S.aligned.fna"
OUTPUT_CSV=f"{DIR_zeqian}/zeqian_16S.aligned.csv"
TREE=f"{DIR_zeqian}/zeqian_16S.tree"
generate_tree(INPUT=INPUT, OUTPUT_ALIGNED=OUTPUT_ALIGNED, OUTPUT_CSV=OUTPUT_CSV, TREE=TREE, threads=48)

17:32:03 [SINA] This is SINA 1.7.2.
Processing: 0 [00:00:28]
Scanning:   0% |      | 1/2224740 [00:00:00 / 00:00:58]
Processing: 0 [00:00:28]
Scanning:   0% |      | 1/2224740 [00:00:00 / 00:02:20]
Processing: 0 [00:00:28]
Scanning:   0% |   | 9797/2224740 [00:00:00 / 00:00:02]
Processing: 0 [00:00:28]
Scanning:   1% |  | 20754/2224740 [00:00:00 / 00:00:02]
Processing: 0 [00:00:28]
Scanning:   2% |  | 33968/2224740 [00:00:00 / 00:00:02]
Processing: 0 [00:00:28]
Scanning:   2% |  | 47181/2224740 [00:00:00 / 00:00:02]
Processing: 0 [00:00:28]
Scanning:   3% |  | 60018/2224740 [00:00:00 / 00:00:02]
Processing: 0 [00:00:28]
Scanning:   3% |  | 73232/2224740 [00:00:00 / 00:00:02]
Processing: 0 [00:00:28]
Scanning:   4% |  | 86444/2224740 [00:00:00 / 00:00:02]
Processing: 0 [00:00:28]
Scanning:   4% |  | 99282/2224740 [00:00:00 / 00:00:02]
Processing: 0 [00:00:28]
Scanning:   5% | | 113225/2224740 [00:00:00 / 00:00:02]
Processing: 0 [00:00:28]
Scanning:   6% | | 127167/2224740 [00:00:00 / 00

##  1.3 Final organization

In [ ]:
# ko
DIR_KOFAMSCAN=f"{DIR_zeqian}/kofamscan"
ko_data={}
for ff in tqdm(os.listdir(DIR_KOFAMSCAN)):
    ko_data[ff.replace('.ko','')]=pd.read_csv(os.path.join(DIR_KOFAMSCAN,ff),sep='\t',header=None,names=['orf','ko'])['ko'].value_counts()
ko_data=pd.DataFrame(ko_data).fillna(0).astype(int)
ko_data=(ko_data>0).astype(int).T # binarize

# growth data
growth_data=pd.read_csv(f"{DIR_zeqian}/raw/zl_growth_data_20220921.csv",index_col=0)

# tree
from ete3 import Tree
tree=Tree(f"{DIR_zeqian}/zeqian_16S.tree",format=1)

# Finalize 
zeqian_final=finalize_data(ko_data, growth_data, tree,remove_prefix=True)
with open(os.path.join(DIR_zeqian,"zeqian.pk"),'wb') as f:
    pickle.dump(zeqian_final,f)

## 1.4 FBA results

In [ ]:
import re

def find_sample_name(ff):
    return re.findall(r"\/?([^_/]*)[_\w]*.csv",ff)[0]

def concatenate(ffs,output):
    growths=[pd.read_csv(ff,index_col=0,names=[find_sample_name(ff)]) for ff in ffs]
    growth_matrix=pd.concat(growths,axis=1)
    return growth_matrix

DIR_fba=f"{DIR_zeqian}/fba_prediction"
DIR_fba_concat=f"{DIR_zeqian}/fba_concatenated"
samples=np.unique([find_sample_name(ff) for ff in os.listdir(DIR_fba)])
thresh=0.05

for suffix in ['','_gapfill','_force_uptake','_gapfill_force_uptake']:
    ffs=[os.path.join(DIR_fba,samples+suffix+'.csv') for samples in samples]
    growth_matrix=concatenate(ffs,os.path.join(DIR_zeqian,'fba_prediction'+suffix+'.csv'))
    growth_matrix.to_csv(os.path.join(DIR_fba_concat,'zeqian_fba'+suffix+'.csv'))
    (growth_matrix>thresh).astype(int).to_csv(os.path.join(DIR_fba_concat,'zeqian_fba'+suffix+f'_binarized_{thresh}.csv'))


# Dataset 2: Matti

Source: SI in https://www.biorxiv.org/content/10.1101/2022.08.04.502823v1.abstract

In [5]:
DIR_matti="/home/zeqianli/project/zeqian/Carbon/data/matti"

## Pre-processing: genome data, 16S

In [ ]:
# Genome: convert xlsx to pickle 
data=pd.read_excel(f"{DIR_matti}/raw/Code & Data for Fundamental metabolic strategies of heterotrophic bacteria/Genomes & annotations/eggnog-results.emapper.annotations.xlsx")

data.columns=data.iloc[3,:].values
data.columns=[data.columns[0][1:],*data.columns[1:]]
data=data[~data['query_name'].str.startswith('#')].copy()

data['KEGG_primary']=data['KEGG_ko'].str.split(',').str[0]
data['root_og']=data['eggNOG OGs'].str.extract(r'root,([\w\d]+)@')


data.to_pickle(f"{DIR_matti}/raw/Code & Data for Fundamental metabolic strategies of heterotrophic bacteria/Genomes & annotations/eggnog.pk")
data.to_csv(f"{DIR_matti}/raw/Code & Data for Fundamental metabolic strategies of heterotrophic bacteria/Genomes & annotations/eggnog.csv",index=False)

16S: Matti data has no 16S data. Need to run barrnap


In [ ]:
# Copy contigs 
import shutil

DIR_assembly_raw=f"{DIR_matti}/raw/Code & Data for Fundamental metabolic strategies of heterotrophic bacteria/Genomes & annotations/SC192 contigs"
DIR_assembly=f"{DIR_matti}/contigs"
for ff in os.listdir(DIR_assembly_raw):
    if ff.endswith('.fsa'):
        shutil.copy(os.path.join(DIR_assembly_raw)ra,ff),os.path.join(DIR_assembly,ff)

In [ ]:
# Run Barrnap

threads=28
DIR_assembly=f"{DIR_matti}/contigs"
DIR_16S=f"{DIR_matti}/16S"

for ff in os.listdir(DIR_assembly):
    if ff.endswith('.fsa'):
        !/home/zeqianli/scratch-midway3/miniconda3/envs/sixteenS/bin/barrnap --kingdom bac --threads {threads} --outseq {os.path.join(DIR_16S, ff.replace('.fsa','.16S.fna'))} {os.path.join(DIR_assembly, ff)}

In [ ]:
# Concatenate barrnap output
concatenate_barrnap_output(f"{DIR_matti}/16S", f"{DIR_matti}/matti_16S.fna",prefix='matti')

In [6]:
# Build tree
INPUT=f"{DIR_matti}/matti_16S.fna"
OUTPUT_ALIGNED=f"{DIR_matti}/matti_16S.aligned.fna"
OUTPUT_CSV=f"{DIR_matti}/matti_16S.aligned.csv"
TREE=f"{DIR_matti}/matti_16S.tree"
generate_tree(INPUT=INPUT, OUTPUT_ALIGNED=OUTPUT_ALIGNED, OUTPUT_CSV=OUTPUT_CSV, TREE=TREE,threads=48)

17:34:14 [SINA] This is SINA 1.7.2.
Processing: 0 [00:00:28]
Scanning:   0% |      | 1/2224740 [00:00:00 / 00:01:22]
Processing: 0 [00:00:28]
Scanning:   0% |      | 1/2224740 [00:00:00 / 00:02:47]
Processing: 0 [00:00:28]
Scanning:   0% |  | 10274/2224740 [00:00:00 / 00:00:02]
Processing: 0 [00:00:28]
Scanning:   1% |  | 20754/2224740 [00:00:00 / 00:00:02]
Processing: 0 [00:00:28]
Scanning:   2% |  | 34323/2224740 [00:00:00 / 00:00:01]
Processing: 0 [00:00:28]
Scanning:   2% |  | 47890/2224740 [00:00:00 / 00:00:02]
Processing: 0 [00:00:28]
Scanning:   3% |  | 60947/2224740 [00:00:00 / 00:00:02]
Processing: 0 [00:00:28]
Scanning:   3% |  | 74171/2224740 [00:00:00 / 00:00:01]
Processing: 0 [00:00:28]
Scanning:   4% |  | 87394/2224740 [00:00:00 / 00:00:02]
Processing: 0 [00:00:28]
Scanning:   4% | | 100004/2224740 [00:00:00 / 00:00:02]
Processing: 0 [00:00:28]
Scanning:   5% | | 113842/2224740 [00:00:00 / 00:00:02]
Processing: 0 [00:00:28]
Scanning:   6% | | 127679/2224740 [00:00:00 / 00

## Organize


In [ ]:
# Genome data

eggnog=pd.read_pickle(f"{DIR_matti}/raw/Code & Data for Fundamental metabolic strategies of heterotrophic bacteria/Genomes & annotations/eggnog.pk")

import re

dir_faa=f"{DIR_matti}/raw/Code & Data for Fundamental metabolic strategies of heterotrophic bacteria/Genomes & annotations/dbcanSC192_v3" 

genes=eggnog.copy()
genes['sample']=None

if genes['query_name'].duplicated().any():
    print("Warning: duplicated query_name")

genes.set_index('query_name',inplace=True)

prefix_sample_map={}
for d in os.listdir(dir_faa):
    sample=re.findall(r'dbcan_(.+).faa',d)[0] # Can't match samples for some eggnog annotations. Probably need to contact Matti for them. Ignore them for now. 
    dbscan=pd.read_csv(os.path.join(dir_faa,d,'overview.txt'),sep='\t')
    prefix=dbscan['Gene ID'].iloc[0].split('_')[0]
    if prefix in prefix_sample_map:
        print("Warning: duplicated prefix")
    prefix_sample_map[prefix]=sample

genes['sample']=genes.index.str.split('_').str[0].map(prefix_sample_map)

ko_data=genes[genes['KEGG_primary'].str.startswith('ko:')].groupby(['sample','KEGG_primary']).size().reset_index().pivot(index='sample',columns='KEGG_primary',values=0).fillna(0).astype(int)
ko_data.columns=ko_data.columns.str.replace('ko:','')

In [ ]:
# Growth data
growth_raw=pd.read_excel(f"{DIR_matti}/raw/Code & Data for Fundamental metabolic strategies of heterotrophic bacteria/Supplementary files/SI Table 3 (growth data).xlsx",sheet_name='growth data',header=1)
growth_data=growth_raw.drop(columns=["kingdom","phylum","class","order","family","genus","species",r"sugar-acid-preference (averaged over 3 experiments, -1=acids, +1=sugars)",r"genome %GC"]).copy()
growth_data.rename(columns={"strain ID":"sample"},inplace=True)
if growth_data['sample'].duplicated().any():
    print("Warning: duplicated sample")
growth_data.set_index('sample',inplace=True)
growth_data=(growth_data>0).astype(int)

In [ ]:
tree=Tree(f"{DIR_matti}/matti_16S.tree",format=1)

In [ ]:
growth_data=growth_data.drop(index=['matti_DSS-3']) # This sample failed 16S annotation

matti_data=finalize_data(ko_data,growth_data,tree,remove_prefix=True)

with open(f"{DIR_matti}/matti.pk",'wb') as f:
    pickle.dump(matti_data,f)

# Dataset 3: BacDive

In [7]:
DIR_bacdive="/home/zeqianli/project/zeqian/Carbon/data/bacdive"

## Download raw data 

In [ ]:
# Downlaod raw data
# In BacDive search section (https://bacdive.dsmz.de/advsearch), filter by NCBI taxon ID and metabolite utilization being available. Download the spreadsheet.
# Note this section needs [BacDive Python API](https://pypi.org/project/bacdive/).

import bacdive

strains=pd.read_csv("download/20221024_bacdive.csv",dtype={'ID':str}).dropna(subset=['ID'])

EMAIL=None # Put your own email
PASSWORD=None # Put your own password
client = bacdive.BacdiveClient(EMAIL,PASSWORD)

ids=strains['ID'].tolist()
raw=[]
MAX_QUERY=100

for i in range(0,len(ids),MAX_QUERY):
    try:
        search=client.search(id=';'.join(ids[i:i+MAX_QUERY]))
        if search:
            result=client.retrieve()
            raw.extend(list(result))
            print("Query success: ",i)
        else:
            print('No result for',ids[i:i+MAX_QUERY])
            raise ValueError
    except Exception as e:
        print(f"Query failed i={i}. {e}")

with open(f"{DIR_bacdive}/raw/raw.pk",'wb') as f:
    pickle.dump(raw, f)

# TODO: id, taxon_id, biosample_id, taxa lookup 

## Parse utilization data

In [ ]:
def get_utilization_data(record,keys=['Chebi-ID','metabolite','utilization activity','kind of utilization tested']):
    # TODO: specify if the data source is api-test or not
    bacdive_id=record['General']['BacDive-ID']
    outs=[]
    try:
        records=record['Physiology and metabolism']['metabolite utilization']
        if isinstance(records,dict):
            records=[records] # Use the first record for now. 
    except KeyError:
        return outs
    
    for r in records:
        out={'id':bacdive_id}
        for k in keys:
            try:
                out[k]=str(r[k])
            except KeyError:
                out[k]=None
        outs.append(out)
    return outs

raw=pd.read_pickle(f"{DIR_bacdive}/raw/raw.pk")
df_utilization=[]
for record in raw:
    df_utilization+=get_utilization_data(record)
df_utilization=pd.DataFrame(df_utilization,dtype=str)

In [ ]:
df_utilization=df_utilization[df_utilization['kind of utilization tested'].isin(['energy source','carbon source'])] # Filter out other kinds of utilization.
df_utilization=df_utilization[df_utilization['utilization activity'].isin(['+','-'])] # Remove uncertain records.
df_utilization['metabolite']=df_utilization['metabolite'].str.replace('D-','').str.replace('L-','') # Remove D- and L- prefix. 
df_utilization['metabolite']=df_utilization['metabolite'].str.replace('\w*sodium\ +','',regex=True).replace('\w*potassium\ +','',regex=True) 
df_utilization['metabolite']=df_utilization['metabolite'].str.replace(r'\(.*\)- *','',regex=True) # braket prefixes. E.g., (-)-quinic acid -> quinic acid, methyl (R)-lactate -> methyl lactate
df_utilization['metabolite']=df_utilization['metabolite'].str.replace(r'\d+ *%','',regex=True) # This changes "1 % sodium lactate" to "lactate" 

df_utilization=df_utilization.groupby(['id','metabolite']).filter(lambda x: x['utilization activity'].nunique()==1) # Remove conflicting records
df_utilization=df_utilization.drop_duplicates(subset=['id','metabolite']) # Remove duplicated records.
df_utilization=df_utilization.groupby('metabolite').filter(lambda x: len(x)>=9) # Minimum 9 samples
df_utilization=df_utilization.pivot(index='id',columns='metabolite',values='utilization activity') # Convert to wide format
df_utilization=df_utilization.replace({'+':1,'-':0}) # Convert to binary

Note: I shortened metabolite names and merged records. Only <1% of records are affected by this change and these records are removed. See SI. 

In [ ]:
df_utilization.to_csv(f'{DIR_bacdive}/bacdive_utilization.csv')
df_utilization.to_pickle(f'{DIR_bacdive}/bacdive_utilization.pk')

## Annotation data from progenomes

In [ ]:
# Parse bacdive entries
def get_genome_info(record,keys=['description','accession','NCBI tax ID']):
    out={'id':record['General']['BacDive-ID']}
    try:
        record=record['Sequence information']['Genome sequences']
        if isinstance(record,list):
            record=record[0] # Use the first record for now. 
    except KeyError:
        for k in keys:
            out[k]=None
        return out
    
    for k in keys:
        try:
            out[k]=str(record[k])
        except KeyError:
            out[k]=None
    return out

raw=pd.read_pickle(f"{DIR_bacdive}/raw/raw.pk")
df_genome=[]
for record in raw:
    df_genome.append(get_genome_info(record))
df_genome=pd.DataFrame(df_genome,dtype=str)
df_genome=df_genome.dropna(subset=['NCBI tax ID']) # This entry is required for progenomes 

In [ ]:
from io import StringIO
import urllib.request

# Get biosample id
def get_ncbi_biosample_id(row):
    #id=row['NCBI tax ID']
    acc=row['accession']
    try:
        out=!datasets summary genome accession {acc}.1
        out=json.load(StringIO(out[0]))['assemblies'][0]['assembly']['biosample_accession']
        return out
    except:
        print("Error querying biosample ID for ",row['id'], acc)
        return None

def download_eggnog(row,overwrite=True,dir_out="/media/zeqian/cf24fe8e-c86d-4712-b58b-9b0f3a718ba91/CarbonUtilization/bacdive/raw/eggnog/"):
    ff_out=os.path.join(dir_out, f"{row['id']}_{row['accession']}_{row['NCBI tax ID']}_{row['biosample']}.csv")
    if os.path.exists(ff_out):
        print("File exists: ",row['id'])
        if not overwrite:
            return
            
    progenomes_eggnog_temp="https://progenomes.embl.de/dumpAnnotation.cgi?p={NCBI_tax_ID}.{Biosample_ID}&t=e&a={NCBI_tax_ID}" 
    url=progenomes_eggnog_temp.format(NCBI_tax_ID=row['NCBI tax ID'],Biosample_ID=row['biosample'])

    try:
        urllib.request.urlretrieve(url,ff_out)
        print("Downloaded ",row['id'])
    except:
        print("Error downloaing eggnog for ",row['id'])
        return None

In [ ]:
# Download from progenomes
timeout=30 

dir_download=f"{DIR_bacdive}/raw/eggnog/"
downloaded=set([x.split('_')[0] for x in os.listdir(dir_download)])
from multiprocessing import Process

def retrieve_biosample_and_download_eggnog(row):
    biosample_id=get_ncbi_biosample_id(row)
    row=row.copy()
    row['biosample']=biosample_id
    download_eggnog(row)


for _, row in df_genome.iterrows():
    if row['id'] in downloaded:
        print("Already downloaded: ",row['id'])
        continue
    
    try:
        p=Process(target=retrieve_biosample_and_download_eggnog,args=(row,))
        p.start()
        p.join(timeout)

        if p.is_alive():
            print("Download timed out: ",row['id'])
            p.terminate()
            p.join()
    except Exception as e:
        print("Error downloading: ",row['id'])
        print(e)


In [ ]:
# Remove invalid downloads

for ff in os.listdir(dir_download):
    min_line_number=2
    ff=os.path.join(dir_download,ff)
    with open(ff) as f:
        if len(f.readlines())<min_line_number:
            print("Removing invalid file: ",ff)
            os.remove(ff)

Some bacdive genomes are duplicate. But for each bacdive genome, each query_name has a unique annotation. 

In [ ]:
# Convert eggnog records to a matrix

ko_matrix={}
eggnog_matrix={}

for ff in tqdm(os.listdir(f"{DIR_bacdive}/raw/eggnog")):
    df_eggnog=pd.read_csv(os.path.join(f"{DIR_bacdive}/raw/eggnog",ff),sep='\t')
    bacdive_id, ncbi_acc0, ncbi_acc1, ncbi_tax_id, biosample_id=ff.replace('.csv','').split('_')
    ncbi_acc=ncbi_acc0+'_'+ncbi_acc1

    df_eggnog['KEGG_primary']=df_eggnog['KEGG_KO'].str.split(',').str[0]
    df_eggnog['root_og']=df_eggnog['EGGNOG_OGS'].str.extract(r'root,([\w\d]+)@')

    ko_matrix[bacdive_id]=df_eggnog[df_eggnog['KEGG_primary'].str.startswith('ko:')].groupby('KEGG_primary').size().to_dict()
    eggnog_matrix[bacdive_id]=df_eggnog.groupby('root_og').size().to_dict()

In [ ]:
ko_matrix

In [ ]:
ko_matrix=pd.DataFrame(ko_matrix).T.fillna(0).astype(int)
ko_matrix.columns=ko_matrix.columns.str.replace('ko:','')

ko_matrix.to_pickle(f"{DIR_bacdive}/bacdive_ko_data.pk")
# eggnog_matrix=pd.DataFrame(eggnog_matrix).T.fillna(0).astype(int)
# eggnog_matrix.to_pickle(os.path.join(dir_download,'eggnog_matrix.pk'))

## 16S data from ENA

#### Download 16S data from ENA database

In [ ]:
!wget -r ftp://ftp.ebi.ac.uk/pub/databases/ena/rRNA/snapshot_latest/con/* -P "/home/zeqianli/project/zeqianli/Kuehn/Carbon/data/bacdive/raw/ena"
!wget -r ftp://ftp.ebi.ac.uk/pub/databases/ena/rRNA/snapshot_latest/std/* -P "/home/zeqianli/project/zeqianli/Kuehn/Carbon/data/bacdive/raw/ena"

In [ ]:
# Concatenate files in the directory that ends with ".fasta.gz"
!cat $(find "/home/zeqianli/project/zeqianli/Kuehn/Carbon/data/bacdive/raw/ena" -name "*.fasta.gz") > "/home/zeqianli/project/zeqianli/Kuehn/Carbon/data/bacdive/raw/ena_rRNA.fasta.gz"

print("Total number of fasta records:")
!zcat "/home/zeqianli/project/zeqianli/Kuehn/Carbon/data/bacdive/raw/ena_rRNA.fasta.gz" | grep -c ">"

In [ ]:
# Process ENA database
from Bio import SeqIO
import gzip

def parse_16S_file(ff):

    def parse_record(record):
        dic={}
        dic['description']=record.description
        dic['seq']=str(record.seq)
        return dic
    
    df=[]
    with gzip.open(ff,'rt') as f:
        for record in tqdm(SeqIO.parse(f,'fasta')):
            df.append(parse_record(record))
    
    return pd.DataFrame(df)


ena_full=parse_16S_file("/home/zeqianli/project/zeqianli/Kuehn/Carbon/data/bacdive/raw/ena_rRNA.fasta.gz")

ena_full['ena_acc']=ena_full['description'].str.split('|').str[1].str.split(':').str[0] # Each ENA accession has a version number (e.g. 12456678.1). Keep both versions (12456678.1 and 12456678).
ena_full['ena_acc_trimmed']=ena_full['ena_acc'].str.split('.').str[0]

ena_full.to_pickle("/home/zeqianli/project/zeqianli/Kuehn/Carbon/data/bacdive/raw/ena_rRNA.pk")

#### Match BacDive 16S accessions with ENA database

In [ ]:
raw=pd.read_pickle(f"{DIR_bacdive}/raw/raw.pk")
sixteenS_acc=[]
for record in tqdm(raw):
    bacdive_id=str(record['General']['BacDive-ID'])
    sixteenS_keys=['description','accession','length','database','NCBI tax ID']

    try:
        sixteenS_info=record['Sequence information']['16S sequences']
    except KeyError:
        print("16S data unavailable for ",bacdive_id)

    if isinstance(sixteenS_info,dict):
        sixteenS_info=[sixteenS_info]

    for info in sixteenS_info:
        dic={'bacdive_id':bacdive_id}
        for key in sixteenS_keys:
            try:
                dic[key]=str(info[key])
            except KeyError:
                continue
        sixteenS_acc.append(dic)

sixteenS_acc=pd.DataFrame(sixteenS_acc)
sixteenS_acc=sixteenS_acc[sixteenS_acc['database']=='ena'] # Use only ENA database for now. 
sixteenS_acc.dropna(subset=['length'],inplace=True) # only one record with a wrong accession ID. 

# Each bacdive record can have multiple ENA accession IDs. Some are better and some are wrong (e.g. https://bacdive.dsmz.de/strain/5223). Select the best record after merging with ENA sequences. 

In [ ]:
# Match accession IDs with ENA database
ena_full=pd.read_pickle(f"{DIR_bacdive}/raw/ena_rRNA.pk")
ena=ena_full[ena_full['description'].str.contains(f'16[sS]',regex=True)] # Keep only 16S sequences
ena=ena[(ena['ena_acc'].isin(sixteenS_acc['accession'])) | (ena['ena_acc_trimmed'].isin(sixteenS_acc['accession']))]

ena['partial']=ena['description'].str.contains('partial',regex=True)
ena['ena_length']=ena['seq'].str.len()
ena['_ind']=ena.index.values
ena=ena.sort_values(by=['ena_acc_trimmed','ena_length','partial','_ind'],ascending=[True,False,False,True]).drop_duplicates(subset='ena_acc_trimmed',keep='first') # If multiple 16S sequences are present for the same accession, keep the non-partial one, and then the longest one. Manually checked and this is ok. If the length in bacdive and length in ENA are different, it's ok. 
ena.columns=ena.columns.map(lambda x: x+'_ena')


In [ ]:
sixteenS_acc['accession_trimmed']=sixteenS_acc['accession'].str.split('.').str[0]
# Merge with bacdive data
# Note that bacdive enc accession number can be either the full number or the trimmed number. I check that in ENA database, each trimmed accession has only one full accession. So trim the bacdive accession and then merge with the ENA trimmed accession is safe. 
bacdive_with_16S=pd.merge(sixteenS_acc,ena[['description_ena','seq_ena','ena_acc_trimmed_ena','partial_ena','ena_length_ena']],left_on='accession_trimmed',right_on='ena_acc_trimmed_ena',how='left') # First, try to merge with the full accession number

bacdive_with_16S=bacdive_with_16S.dropna(subset=['seq_ena']) # Drop records with no ENA sequences. Checked that there is indeed problems on BacDive (e.g. https://bacdive.dsmz.de/strain/160325)
bacdive_with_16S=bacdive_with_16S.sort_values(by=['bacdive_id','ena_length_ena'],ascending=[True,False]).drop_duplicates(subset=['bacdive_id'],keep='first') # If multiple 16S sequences are present, keep the longest one. Probably ok. In some records (https://bacdive.dsmz.de/strain/9984), the longest record is not the same reference as other ones. I don't see why this is not ok. 


In [ ]:
# Write to a fasta file
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

seqs=[]
for _, row in bacdive_with_16S.iterrows():
    seqs.append(SeqRecord(Seq(row['seq_ena']),id='bacdive_'+row['bacdive_id'],description=row['description_ena']))

SeqIO.write(seqs,f"{DIR_bacdive}/bacdive_16S.fna",'fasta')

#### Construct tree

In [8]:
# Build tree
INPUT=f"{DIR_bacdive}/bacdive_16S.fna"
OUTPUT_ALIGNED=f"{DIR_bacdive}/bacdive_16S.aligned.fna"
OUTPUT_CSV=f"{DIR_bacdive}/bacdive_16S.aligned.csv"
TREE=f"{DIR_bacdive}/bacdive_16S.tree"
generate_tree(INPUT=INPUT, OUTPUT_ALIGNED=OUTPUT_ALIGNED,OUTPUT_CSV=OUTPUT_CSV, TREE=TREE,threads=48)

17:36:48 [SINA] This is SINA 1.7.2.
Processing: 0 [00:00:28]
Scanning:   0% |      | 1/2224740 [00:00:00 / 00:01:14]
Processing: 0 [00:00:28]
Scanning:   0% |      | 1/2224740 [00:00:00 / 00:02:41]
Processing: 0 [00:00:28]
Scanning:   0% |   | 8676/2224740 [00:00:00 / 00:00:02]
Processing: 0 [00:00:28]
Scanning:   1% |  | 17350/2224740 [00:00:00 / 00:00:03]
Processing: 0 [00:00:28]
Scanning:   1% |  | 22774/2224740 [00:00:00 / 00:00:03]
Processing: 0 [00:00:28]
Scanning:   1% |  | 28500/2224740 [00:00:00 / 00:00:03]
Processing: 0 [00:00:28]
Scanning:   2% |  | 38202/2224740 [00:00:00 / 00:00:03]
Processing: 0 [00:00:28]
Scanning:   2% |  | 47903/2224740 [00:00:00 / 00:00:03]
Processing: 0 [00:00:28]
Scanning:   3% |  | 59020/2224740 [00:00:00 / 00:00:03]
Processing: 0 [00:00:28]
Scanning:   3% |  | 70135/2224740 [00:00:00 / 00:00:03]
Processing: 0 [00:00:28]
Scanning:   4% |  | 78539/2224740 [00:00:00 / 00:00:03]
Processing: 0 [00:00:28]
Scanning:   4% |  | 83850/2224740 [00:00:00 / 00

## Final organize

In [ ]:
# save all data
 
ko_data=pd.read_pickle(f"{DIR_bacdive}/bacdive_ko_data.pk")
growth_data=pd.read_pickle(f"{DIR_bacdive}/bacdive_utilization.pk")
tree=Tree(f"{DIR_bacdive}/bacdive_16S.tree",format=1)

bacdive_final=finalize_data(ko_data,growth_data,tree,remove_prefix=True)
with open(f"{DIR_bacdive}/bacdive.pk",'wb') as f:
    pickle.dump(bacdive_final,f)


# Merged data

In [ ]:
DIR_data="/home/zeqianli/project/zeqian/Carbon/data"


In [ ]:
!cat {DIR_data}/zeqian/zeqian_16S.fna {DIR_data}/matti/matti_16S.fna {DIR_data}/bacdive/bacdive_16S.fna > {DIR_data}/merged/merged_16S.fna

In [ ]:
# Build tree

INPUT=f"{DIR_data}/merged/merged_16S.fna"
OUTPUT_ALIGNED=f"{DIR_data}/merged/merged_16S.aligned.fna"
TREE=f"{DIR_data}/merged/merged_16S.tree"

generate_tree(INPUT=INPUT, OUTPUT_ALIGNED=OUTPUT_ALIGNED, TREE=TREE,threads=28)

In [ ]:
ko_data_merged=[]
for dataset in ['zeqian','matti','bacdive']:
    data=pd.read_pickle(f"{DIR_data}/{dataset}/{dataset}.pk")
    data['ko_data'].index=dataset+'_'+data['ko_data'].index
    ko_data_merged.append(data['ko_data'])

ko_data_merged=pd.concat(ko_data_merged,axis=0)
ko_data_merged.fillna(0,inplace=True)

# TODO: utilization data. Need to figure how carbon renaming. 

tree=Tree(f"{DIR_data}/merged/merged_16S.tree",format=1)

merged_final=finalize_data(ko_data_merged,growth_data,tree,remove_prefix=False)

with open(f"{DIR_data}/merged/merged.pk",'wb') as f:
    pickle.dump(merged_final,f)